In [ ]:
pip install pandas sqlalchemy pymysql

In [1]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError

In [7]:
from sqlalchemy import create_engine

# Correctly formatted database connection details
DATABASE_TYPE = 'mysql'
DBAPI = 'mysqlconnector'  # Assuming you're using mysqlconnector
HOST = 'fashionretail.c3q4cw8i0vbc.us-east-2.rds.amazonaws.com'
USER = 'admin'
PASSWORD = 'bsan-6080-project'
DATABASE = 'fashion_retail_sales'

# Correctly create the DATABASE_URL
DATABASE_URL = f"{DATABASE_TYPE}+{DBAPI}://{USER}:{PASSWORD}@{HOST}/{DATABASE}"

# Create the SQLAlchemy engine
engine = create_engine(DATABASE_URL, echo=False)

# Test the connection
try:
    with engine.connect() as conn:
        print("Successfully connected to the database!")
except Exception as e:
    print(f"An error occurred: {e}")


Successfully connected to the database!


In [9]:
create_customers_table_sql = """
CREATE TABLE IF NOT EXISTS Customers (
    CustomerID INT AUTO_INCREMENT PRIMARY KEY,
    CustomerReferenceID INT UNIQUE NOT NULL
);
"""

In [8]:

create_purchases_table_sql = """
CREATE TABLE IF NOT EXISTS Purchases (
    PurchaseID INT AUTO_INCREMENT PRIMARY KEY,
    CustomerReferenceID INT NOT NULL,
    ItemPurchased VARCHAR(255),
    PurchaseAmountUSD DECIMAL(10, 2),
    DatePurchase DATE,
    ReviewRating DECIMAL(3, 1),
    PaymentMethod VARCHAR(50),
    FOREIGN KEY (CustomerReferenceID) REFERENCES Customers(CustomerReferenceID)
);
"""

In [10]:
with engine.begin() as conn:
    conn.execute(create_customers_table_sql)
    conn.execute(create_purchases_table_sql)

In [11]:
csv_file_path = 'Fashion_Retail_Sales_Clean.csv'  # Update this path
df = pd.read_csv(csv_file_path)

In [12]:
customers_df = df[['Customer Reference ID']].drop_duplicates().rename(columns={'Customer Reference ID': 'CustomerReferenceID'})
customers_df.to_sql('Customers', con=engine, if_exists='append', index=False, method='multi')

-1

In [13]:
purchases_df = df.drop(columns=['Unnamed: 0']).rename(columns={
    'Customer Reference ID': 'CustomerReferenceID',
    'Item Purchased': 'ItemPurchased',
    'Purchase Amount (USD)': 'PurchaseAmountUSD',
    'Date Purchase': 'DatePurchase',
    'Review Rating': 'ReviewRating',
    'Payment Method': 'PaymentMethod'
})

In [14]:
purchases_df.to_sql('Purchases', con=engine, if_exists='append', index=False, method='multi')

print("Data insertion complete.")

Data insertion complete.
